In [ ]:
import tensorflow as tf
import scipy as sp
from scipy import ndimage
import numpy as np
import os

In [ ]:
# The inputs and outptus are the maps downloaded from https://www.nnvl.noaa.gov/view/globaldata.html
# The inputs to the model are Land Surface Temperature, Moisture and Longwave Energy. 
# The output is the average precipitation map. 
# All maps are in the average weekly format. The idea is to find correlation between the input maps and 
# the output. 

input_dirs = ['./grayscale/Land_Temp/', './grayscale/Moisture/', './grayscale/Longwave_Energy/'];
input_labels = ['land', 'moisture', 'longwave_energy'];
input_dict = {}

# Process the input : input_metrics will contain a 2048 x 4096 image with 9 channels (corresponding to 3 inputs)

for i, path in enumerate(input_dirs):
    label = input_labels[i];
    input_dict[label] = [];
    
    for file in sorted(os.listdir(path)):
        input_dict[label].append(sp.ndimage.imread(path + file))

input_metrics = input_dict[input_labels[0]];
for label in (input_labels[1:]):
    input_metrics = np.concatenate((input_metrics, input_dict[label]), axis=3);

# Process the output : output_metric will contain a 2048 x 4096 image with 3 channels
output_metric = [];
path = './grayscale/Rain/'
for file in sorted(os.listdir(path)):
    output_metric.append(sp.ndimage.imread(path + file))

output_metric = np.array(output_metric);

samples = input_metrics.shape[0];

In [ ]:
# Function to obtain the next batch based on the input size and the batch size
def next_batch(batch_size):
    
    idx = (np.random.randint(samples,size=(batch_size,)));
    return [input_metrics[idx], output_metric[idx]]

In [ ]:
# Building a simple CNN model that looks like an auto-encoder. This is the section to change for a new model.

def conv_net(x, weights, biases):
    
    x = tf.reshape(x, shape=[-1, 2048, 4096, 9], name='reshape_x');
    x = tf.cast(x, tf.float32) 
    
    # Encoder
    # 2048x4096x9 -> 1024x2048x32
    conv1 = tf.nn.relu(tf.contrib.layers.conv2d(x, 32, [8, 8], stride=2, padding='SAME'))
    # 1024x2048x32 -> 512x1024x16
    conv2 = tf.nn.relu(tf.contrib.layers.conv2d(conv1, 16, [8, 8], stride=2, padding='SAME'))
    # 512x1024x16 -> 256x512x8
    conv3 = tf.nn.relu(tf.contrib.layers.conv2d(conv2, 8, [8, 8], stride=2, padding='SAME'))
    
    # Decoder
    # 256x512x8 -> 512x1024x16
    conv4 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv3, 16, [8, 8], stride=2, padding='SAME'))
    # 512x1024x16 -> 1024x2048x32
    conv5 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv4, 32, [8, 8], stride=2, padding='SAME'))
    # 1024x2048x32 -> 2048x4096x3
    conv6 = tf.nn.relu(tf.contrib.layers.conv2d_transpose(conv5, 3, [8, 8], stride=2, padding='SAME'))
    
    return conv6;

Y = tf.placeholder(tf.float32, shape=(None,2048,4096,3));
X = tf.placeholder(tf.float32, shape=[None,2048,4096,9]);

optimizer = tf.train.AdamOptimizer(learning_rate=0.1);

out = conv_net(X, weights, biases);
loss_op = tf.reduce_sum(tf.multiply(Y-out,Y-out));
train_op = optimizer.minimize(loss_op);

In [ ]:
num_epochs = 100;
batch_size = 1;
num_batches = int(samples / batch_size);

with tf.Session() as sess:
    
    # Run the initializer
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        print(epoch)
        for i in range(num_batches):
            x_train, y_train = next_batch(batch_size);
            sess.run([train_op], feed_dict={X: x_train, Y: y_train});            
        
        loss = sess.run([loss_op], feed_dict={X: x_train, Y: y_train});
        print(epoch, loss);